# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install and Import

In [ ]:
%cd /content
# Training
!gdown 1JHiDLlCS1D4en8i5ImTrMMFI5veIhHQ4
!gdown 1a-vKC9HlZHWee60W51v8BYAVKoL6QqPK
# Testing
!gdown 1JHiDLlCS1D4en8i5ImTrMMFI5veIhHQ4

/content
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1JHiDLlCS1D4en8i5ImTrMMFI5veIhHQ4 

Downloading...
From: https://drive.google.com/uc?id=1a-vKC9HlZHWee60W51v8BYAVKoL6QqPK
To: /content/test_train.csv
100% 8.14M/8.14M [00:00<00:00, 44.3MB/s]
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1JHiDLlCS1D4en8i5ImTrMMFI5veIhHQ4 



In [ ]:
%cd /content
!pip install datasets
!pip install -U accelerate
!pip install -U transformers
!git clone 'https://github.com/vncorenlp/VnCoreNLP.git'
!pip install py_vncorenlp

/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 259, done.
r

In [ ]:
import re
import pandas as pd
import numpy as np

# Training Dataset

## Read Data

In [ ]:
# Training Data
train_df = pd.read_csv('/content/drive/MyDrive/Data Science Challenge 2023/Dataset/train-multilingual.csv', usecols = ['evidence', 'claim', 'verdict'])
train_df.columns = ['claim', 'label', 'evidence']

#Testing for train
test_df = pd.read_csv('/content/test_train.csv', usecols = ['evidence', 'claim', 'verdict'])
test_df.columns = ['claim', 'label', 'evidence']

In [ ]:
train_df.shape, test_df.shape

((26967, 3), (1000, 3))

## Processing

### Encoding Labels

In [ ]:
label2id = {
              'SUPPORTED' : 0,
              'NEI' : 1,
              'REFUTED' : 2
           }

id2label = {
              0 : 'SUPPORTED',
              1 : 'NEI',
              2 : 'REFUTED'
           }

train_df['label'] = [label2id[label] for label in train_df['label']]
test_df['label'] = [label2id[label] for label in test_df['label']]

### Clean data

In [ ]:
def processing_para(para):
  para = para.lower()
  para = re.sub('[^\d\w\s\./]', ' ', para)
  para = re.sub('\s{2}', ' ', para)
  para = ' '.join(para.split('\n'))
  corpus = para.split('.')
  corpus = [sentence.strip() for sentence in corpus]
  corpus = [sentence for sentence in corpus if sentence != '']
  return corpus

def processing_sen(text):
  text = text.lower()
  text = re.sub('[^\d\w\s\./]', ' ', text)
  text = re.sub('\s{2}', ' ', text)
  text = ' '.join(text.split('\n'))
  text = text.strip()
  return text

In [ ]:
train_df['evidence'] = train_df['evidence'].apply(processing_sen)
train_df['claim'] = train_df['claim'].apply(processing_sen)

test_df['evidence'] = test_df['evidence'].apply(processing_sen)
test_df['claim'] = test_df['claim'].apply(processing_sen)

## Word Segmentation

In [ ]:
import py_vncorenlp
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators = ["wseg"], save_dir = '/content/VnCoreNLP')

In [ ]:
def word_segmentation_text(text):
  return rdrsegmenter.word_segment(text)[0]
def word_segmentation_corpus(corpus):
  return [word_segmentation_text(text) for text in corpus]

In [ ]:
train_df[train_df['evidence'] == '']

,claim,label,evidence
20951,khoảng 1.600 đại biểu đã đến tham dự đại hội đ...,0,
20991,trung tâm hồi sức covid 19 được thành lập tại ...,0,


In [ ]:
train_df = train_df.drop(index = [20951, 20991]).reset_index(drop = True)

In [ ]:
train_df['evidence'] = train_df['evidence'].apply(word_segmentation_text)
train_df['claim'] = train_df['claim'].apply(word_segmentation_text)

test_df['evidence'] = test_df['evidence'].apply(word_segmentation_text)
test_df['claim'] = test_df['claim'].apply(word_segmentation_text)

## Create Dataset

In [ ]:
from datasets import DatasetDict, Dataset, DatasetDict, load_dataset, load_metric
dataset = DatasetDict({
    "train" : Dataset.from_pandas(train_df),
    "test" : Dataset.from_pandas(test_df)
})

## Tokenize and Load Model

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

def tokenize_function(examples):
    return tokenizer(examples["claim"], examples["evidence"], padding = "max_length", truncation = True, return_tensors="pt")

tokenized_datasets = dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/26965 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_dataset = tokenized_datasets["train"].shuffle(seed = 42)
test_dataset = tokenized_datasets["test"].shuffle(seed = 42)

### Load Base-Model

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels = 3)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Load Trained-Model

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Data Science Challenge 2023/Checkpoint/checkpoint-3000", num_labels = 3)

In [ ]:
import math
import os
import warnings
from dataclasses import dataclass
from typing import List, Optional, Tuple, Union

import torch
import torch.utils.checkpoint
from torch import nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.activations import ACT2FN
from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    BaseModelOutputWithPoolingAndCrossAttentions,
    CausalLMOutputWithCrossAttentions,
    MaskedLMOutput,
    MultipleChoiceModelOutput,
    NextSentencePredictorOutput,
    QuestionAnsweringModelOutput,
    SequenceClassifierOutput,
    TokenClassifierOutput,
)
from transformers.modeling_utils import PreTrainedModel
from transformers.pytorch_utils import apply_chunking_to_forward, find_pruneable_heads_and_indices, prune_linear_layer
from transformers.utils import (
    ModelOutput,
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    logging,
    replace_return_docstrings,
)

In [ ]:
class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.bert = BertModel(config)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        # Initialize weights and apply final processing
        self.post_init()

    @add_start_docstrings_to_model_forward(BERT_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
    @add_code_sample_docstrings(
        checkpoint=_CHECKPOINT_FOR_SEQUENCE_CLASSIFICATION,
        output_type=SequenceClassifierOutput,
        config_class=_CONFIG_FOR_DOC,
        expected_output=_SEQ_CLASS_EXPECTED_OUTPUT,
        expected_loss=_SEQ_CLASS_EXPECTED_LOSS,
    )
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

NameError: ignored

# Create Trainer and Train

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(do_train = True,
                                  do_eval = True,
                                  evaluation_strategy = 'epoch',
                                  learning_rate = 2e-5,
                                  num_train_epochs = 12,
                                  gradient_accumulation_steps = 2,
                                  per_device_train_batch_size = 32,
                                  per_device_eval_batch_size = 32,
                                  logging_dir='/content/drive/MyDrive/Checkpoint/logs',
                                  output_dir = "/content/drive/MyDrive/Checkpoint/test_trainer",
                                  logging_steps = 20,
                                  weight_decay=0.01
                                  )

In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("accuracy")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis = -1)
    return {
              "precision" : metric1.compute(predictions = predictions, references = labels, average = 'weighted')['precision'],
              "recall" : metric2.compute(predictions = predictions, references = labels, average = 'weighted')['recall'],
              "accuracy" : metric3.compute(predictions = predictions, references = labels)['accuracy']
            }

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,Accuracy
0,0.313800,0.278319,0.898654,0.889000,0.889000
2,0.167800,0.081779,0.972622,0.972000,0.972000
4,0.047600,0.031179,0.990027,0.990000,0.990000
6,0.033800,0.022735,0.993055,0.993000,0.993000
8,0.008800,0.015333,0.996034,0.996000,0.996000
10,0.005300,0.012327,0.996009,0.996000,0.996000
11,0.000600,0.011060,0.997002,0.997000,0.997000


<ipython-input-20-603395dae32b>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric1 = load_metric("precision")


TrainOutput(global_step=5052, training_loss=0.09329230974683492, metrics={'train_runtime': 13130.0016, 'train_samples_per_second': 24.644, 'train_steps_per_second': 0.385, 'total_flos': 4.252004920180685e+16, 'train_loss': 0.09329230974683492, 'epoch': 11.99})

# Public Test

## Read Data

In [ ]:
%cd /content
!gdown 1BhtMBxKLponTKWW8vmNCy0ypi3hlmwNw

/content
Downloading...
From: https://drive.google.com/uc?id=1BhtMBxKLponTKWW8vmNCy0ypi3hlmwNw
To: /content/ise-dsc01-public-test-offcial.json
100% 19.3M/19.3M [00:00<00:00, 77.3MB/s]


In [ ]:
public_test = pd.read_csv('/content/drive/MyDrive/Data Science Challenge 2023/Dataset/public-Multilingual.csv')
public_test

,Unnamed: 0,context,claim,evidence,corpus
0,0,"Ngày 13/3, PGS.TS Phạm Cẩm Phương, Giám đốc Tr...",nên kiểm tra gene trẻ sơ sinh khi gia đình có ...,tùy trường hợp bác sĩ sẽ chỉ định xét nghiệm g...,"['ngày 13/3 pgs', 'ts phạm cẩm phương giám đốc..."
1,1,"STEM bao gồm lĩnh vực khoa học (Science), công...",mba tập trung vào kỹ năng kinh doanh quản lý v...,tuy nhiên mba tập trung vào kỹ năng kinh doanh...,['stem bao gồm lĩnh vực khoa học science công...
2,2,Trong khi tranh cãi còn chưa dứt về số tài liệ...,cuộc tranh cãi còn chưa dứt về số tài liệu mật...,trong khi tranh cãi còn chưa dứt về số tài liệ...,['trong khi tranh cãi còn chưa dứt về số tài l...
3,3,"Nằm trên đường Nguyễn Biểu, quận 5, tiệm há cả...",tiệm há cảo của anh hùng có 6 vị nhân được làm...,tiệm há cảo của anh hùng có 8 vị nhân được làm...,['nằm trên đường nguyễn biểu quận 5 tiệm há cả...
4,4,"Năm nay, thời tiết nắng nhiều nhưng mưa ít khi...",cách trung tâm thủ đô chừng 100km công viên th...,đầm sen công viên thực vật cảnh việt nam cách ...,['năm nay thời tiết nắng nhiều nhưng mưa ít kh...
...,...,...,...,...,...
4789,4789,Cảnh sát cho hay trách nhiệm truy tố và xét xử...,theo truyền thông tây ban nha thì hai công dân...,trước đó truyền thông tây ban nha dẫn thông bá...,['cảnh sát cho hay trách nhiệm truy tố và xét ...
4790,4790,1. Thiết kế taxi bay an toàn và hiệu quả\n\nSử...,geos có thể phục vụ nghiên cứu hệ thống đại dư...,geos có thể phục vụ nghiên cứu hệ thống đại dư...,"['1', 'thiết kế taxi bay an toàn và hiệu quả s..."
4791,4791,"Tối 1/6, ông Đỗ Huy Thông - Phó Chủ tịch UBND ...",trong hành trình du khách chỉ được ngắm vịnh h...,trong hành trình du khách sẽ được ngắm vịnh hạ...,['tối 1/6 ông đỗ huy thông phó chủ tịch ubnd ...
4792,4792,"Các nhân viên đã rải bột mì để thử truy dấu ""k...",họ không lắp đặt những camera đặc biệt được th...,họ cũng lắp đặt những camera đặc biệt được thi...,['các nhân viên đã rải bột mì để thử truy dấu ...


## Processing Data

In [ ]:
public_test['evidence'] = public_test['evidence'].apply(processing_sen)
public_test['claim'] = public_test['claim'].apply(processing_sen)

public_test['evidence'] = public_test['evidence'].apply(word_segmentation_text)
public_test['claim'] = public_test['claim'].apply(word_segmentation_text)

In [ ]:
from datasets import DatasetDict, Dataset, load_dataset, load_metric
public_test_dataset = DatasetDict({
    "test" : Dataset.from_pandas(public_test)
})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

def tokenize_function(examples):
    return tokenizer(examples["claim"], examples["evidence"], padding = "max_length", truncation=True)

tokenized_public_test_dataset = public_test_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/4794 [00:00<?, ? examples/s]

In [ ]:
tokenized_public_test_dataset = public_test_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/4794 [00:00<?, ? examples/s]

In [ ]:
public_test_dataset = tokenized_public_test_dataset["test"]
print(public_test_dataset)

Dataset({
    features: ['Unnamed: 0', 'context', 'claim', 'evidence', 'corpus', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4794
})


In [ ]:
pred = trainer.predict(public_test_dataset)
label_pred = np.argmax(pred.predictions, axis = -1)
label_pred

array([1, 1, 0, ..., 2, 2, 2])

In [ ]:
len(label_pred)

4794

In [ ]:
pred = [id2label[id] for id in label_pred]

In [ ]:
pred

['NEI',
 'NEI',
 'SUPPORTED',
 'REFUTED',
 'REFUTED',
 'REFUTED',
 'SUPPORTED',
 'REFUTED',
 'NEI',
 'REFUTED',
 'NEI',
 'NEI',
 'SUPPORTED',
 'NEI',
 'NEI',
 'NEI',
 'SUPPORTED',
 'NEI',
 'NEI',
 'SUPPORTED',
 'SUPPORTED',
 'REFUTED',
 'NEI',
 'NEI',
 'REFUTED',
 'REFUTED',
 'NEI',
 'SUPPORTED',
 'REFUTED',
 'NEI',
 'REFUTED',
 'REFUTED',
 'SUPPORTED',
 'SUPPORTED',
 'REFUTED',
 'REFUTED',
 'NEI',
 'NEI',
 'REFUTED',
 'SUPPORTED',
 'NEI',
 'SUPPORTED',
 'NEI',
 'REFUTED',
 'NEI',
 'NEI',
 'REFUTED',
 'REFUTED',
 'NEI',
 'REFUTED',
 'NEI',
 'NEI',
 'REFUTED',
 'REFUTED',
 'REFUTED',
 'NEI',
 'REFUTED',
 'SUPPORTED',
 'NEI',
 'REFUTED',
 'NEI',
 'NEI',
 'REFUTED',
 'SUPPORTED',
 'REFUTED',
 'NEI',
 'NEI',
 'NEI',
 'SUPPORTED',
 'NEI',
 'NEI',
 'SUPPORTED',
 'REFUTED',
 'NEI',
 'NEI',
 'REFUTED',
 'SUPPORTED',
 'NEI',
 'SUPPORTED',
 'SUPPORTED',
 'NEI',
 'SUPPORTED',
 'NEI',
 'REFUTED',
 'NEI',
 'NEI',
 'REFUTED',
 'REFUTED',
 'SUPPORTED',
 'NEI',
 'REFUTED',
 'REFUTED',
 'SUPPORTED',
 '

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Data Science Challenge 2023/Dataset/public-Multilingual.csv')
data = data[['evidence']]
data['verdict'] = pred

In [ ]:
data

In [ ]:
train_data = pd.read_json("/content/ise-dsc01-public-test-offcial.json")
train_data

,29706,32454,41947,47909,46798,47255,41443,39120,32538,46995,...,38980,46238,41410,39035,42634,41870,47073,46617,46410,46737
context,"Ngày 13/3, PGS.TS Phạm Cẩm Phương, Giám đốc Tr...","STEM bao gồm lĩnh vực khoa học (Science), công...",Trong khi tranh cãi còn chưa dứt về số tài liệ...,"Nằm trên đường Nguyễn Biểu, quận 5, tiệm há cả...","Năm nay, thời tiết nắng nhiều nhưng mưa ít khi...",Đây là năm thứ 2 cuộc thi Sáng kiến Khoa học (...,Mì sụa là một trong những món ăn truyền thống ...,Trong kỳ nghỉ lễ kéo dài 5 ngày dịp 30/4 - 1/5...,Nguyễn Ngọc Thiện là học sinh lớp 7 tại trường...,Đây là năm thứ hai VnExpress tổ chức cuộc thi ...,...,"Bản Nậm Nghiệp, xã Ngọc Chiến, huyện Mường La ...","Độc giả Cao Chu Phương, 28 tuổi, ở Biên Hòa, k...","Kat Kamalani, một cựu tiếp viên hàng không, ti...",Mùa hoa anh đào ở Praha bắt đầu vào tháng 3 và...,Trả lời:\n\nDa đàn ông thường dày và nhiều dầu...,Cảnh sát cho hay trách nhiệm truy tố và xét xử...,1. Thiết kế taxi bay an toàn và hiệu quả\n\nSử...,"Tối 1/6, ông Đỗ Huy Thông - Phó Chủ tịch UBND ...","Các nhân viên đã rải bột mì để thử truy dấu ""k...","Theo tờ Straits Times, Singapore đang bước vào..."
claim,Nên kiểm tra gene trẻ sơ sinh khi gia đình có ...,"mba tập trung vào kỹ năng kinh doanh, quản lý ...",Cuộc tranh cãi còn chưa dứt về số tài liệu mật...,"Tiệm há cảo của anh Hùng có 6 vị nhân, được là...","Cách trung tâm Thủ đô chừng 100km, công viên T...",Yêu cầu giải pháp/sản phẩm dự thi được mô tả b...,Món ăn truyền thống của người Hoa là mì sụa,Trong các ngày cuối tuần từ 14/4 đến dịp lễ 30...,thiện đã chia sẻ kinh nghiệm học tập với hơn 1...,Ngoài ra cuộc thi cũng chọn và trao cho tác gi...,...,Xe của dân địa phương ngồi rất thoải mái,"Sau khi đáp xuống Leh, thủ phủ vùng Ladakh, và...",Hành khách và tiếp viên đều có thể bị thương,Mùa hoa anh đào ở Praha bắt đầu vào tháng 3 và...,"Rửa, làm sạch da là bước cơ bản đầu tiên và qu...",Theo truyền thông Tây Ban Nha thì hai công dân...,GEOS có thể phục vụ nghiên cứu hệ thống đại dư...,"Trong hành trình, du khách chỉ được ngắm vịnh ...",Họ không lắp đặt những camera đặc biệt được th...,"Chủ quan, ông Steven Qin cho biết mình đang tặ..."


In [ ]:
data.index = train_data.columns

In [ ]:
data = data[['verdict', 'evidence']]
data

,verdict,evidence
29706,NEI,tùy trường hợp bác sĩ sẽ chỉ định xét nghiệm g...
32454,NEI,tuy nhiên mba tập trung vào kỹ năng kinh doanh...
41947,SUPPORTED,trong khi tranh cãi còn chưa dứt về số tài liệ...
47909,REFUTED,tiệm há cảo của anh hùng có 8 vị nhân được làm...
46798,REFUTED,đầm sen công viên thực vật cảnh việt nam cách ...
...,...,...
41870,SUPPORTED,trước đó truyền thông tây ban nha dẫn thông bá...
47073,REFUTED,geos có thể phục vụ nghiên cứu hệ thống đại dư...
46617,REFUTED,trong hành trình du khách sẽ được ngắm vịnh hạ...
46410,REFUTED,họ cũng lắp đặt những camera đặc biệt được thi...


In [ ]:
evi_lst = []
for _, row in data.iterrows():
  if (row['verdict'] == 'NEI'):
    evi_lst.append('')
  else:
    evi_lst.append(row['evidence'])

data['evidence'] = evi_lst
data

,verdict,evidence
29706,NEI,
32454,NEI,
41947,SUPPORTED,trong khi tranh cãi còn chưa dứt về số tài liệ...
47909,REFUTED,tiệm há cảo của anh hùng có 8 vị nhân được làm...
46798,REFUTED,đầm sen công viên thực vật cảnh việt nam cách ...
...,...,...
41870,SUPPORTED,trước đó truyền thông tây ban nha dẫn thông bá...
47073,REFUTED,geos có thể phục vụ nghiên cứu hệ thống đại dư...
46617,REFUTED,trong hành trình du khách sẽ được ngắm vịnh hạ...
46410,REFUTED,họ cũng lắp đặt những camera đặc biệt được thi...


In [ ]:
result = data.to_json('/content/public_result-multilingual.json', orient="index", force_ascii = False)

In [ ]:
train_data = pd.read_json('/content/public_result-multilingual.json')
data = np.transpose(train_data.values)
train_df = pd.DataFrame(data)
train_df.head()

,0,1
0,NEI,
1,NEI,
2,SUPPORTED,trong khi tranh cãi còn chưa dứt về số tài liệ...
3,REFUTED,tiệm há cảo của anh hùng có 8 vị nhân được làm...
4,REFUTED,đầm sen công viên thực vật cảnh việt nam cách ...
